In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from pydataset import data
from datetime import date
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')
from env import get_db_url, user, password, host
import acquire
import prepare

In [2]:
# 2. Given confusion matrix:
# What is a false positive? Predicting a cat when it is actually a dog
# What is a false negative? Predicting a dog and it was actually a cat
Accuracy= (46 + 13) / (46 + 13 + 7 + 34)

In [3]:
Accuracy

0.59

In [4]:
Recall = 46 / (46 + 13)

In [5]:
Recall

0.7796610169491526

In [6]:
Precision = 46 / (46 + 7)

In [7]:
Precision

0.8679245283018868

In [8]:
# This model, if the + is dog, has a high precision outcome, fairly high recall, and moderate accuracy. 

In [9]:
# 3. C3 Rubber-duck manufacturing
# Importing the .csv
c3 = pd.read_csv('c3.csv')

In [10]:
c3

,actual,model1,model2,model3
0,No Defect,No Defect,Defect,No Defect
1,No Defect,No Defect,Defect,Defect
2,No Defect,No Defect,Defect,No Defect
3,No Defect,Defect,Defect,Defect
4,No Defect,No Defect,Defect,No Defect
...,...,...,...,...
195,No Defect,No Defect,Defect,Defect
196,Defect,Defect,No Defect,No Defect
197,No Defect,No Defect,No Defect,No Defect
198,No Defect,No Defect,Defect,Defect


In [11]:
m1_cross = pd.crosstab(c3.model1, c3.actual)

In [12]:
m2_cross = pd.crosstab(c3.model2, c3.actual)

In [13]:
m3_cross = pd.crosstab(c3.model3, c3.actual)

In [14]:
m1_acc = (c3.actual == c3.model1).mean()

In [15]:
m2_acc = (c3.actual == c3.model2).mean()
m3_acc = (c3.actual == c3.model3).mean()

In [16]:
m1_acc, m2_acc, m3_acc

(0.95, 0.56, 0.555)

In [17]:
# precision - assuming our + is defect
m1_cross, m2_cross, m3_cross

(actual     Defect  No Defect
 model1                      
 Defect          8          2
 No Defect       8        182,
 actual     Defect  No Defect
 model2                      
 Defect          9         81
 No Defect       7        103,
 actual     Defect  No Defect
 model3                      
 Defect         13         86
 No Defect       3         98)

In [18]:
subset = c3[c3.model1 == 'Defect']
(subset.model1 == subset.actual).mean()

0.8

In [19]:
subset = c3[c3.model2 == 'Defect']
(subset.model2 == subset.actual).mean()

0.1

In [20]:
subset = c3[c3.model3 == 'Defect']
(subset.model3 == subset.actual).mean()

0.13131313131313133

In [21]:
# recall - assumiong + is defect
subset = c3[c3.actual == 'Defect']
(subset.model1 == subset.actual).mean()

0.5

In [22]:
(subset.model2 == subset.actual).mean()

0.5625

In [23]:
(subset.model3 == subset.actual).mean()

0.8125

In [24]:
# Identifying as many of the ducks that have a defect as possible

# M1 is the only model that has a useful accuracy if the concern accounts for lost time/expense of FP. It also has 
# much better precision performance, which is best for mass production efficiency of ducks. Guaging for 
# accuracy and precision makes sense here, and should go with M1. 

# Defects = Trip to Hawaii

# Here you would want to use recall to reduce the high cost of FN (missed defective ducks off assembly line)
# This would mean that M3 would probably be the best use given the extreme cost of missing defective ducks. 

In [27]:
# 4. Gives You Paws
paws = pd.read_csv('gives_you_paws.csv')

In [28]:
paws

,actual,model1,model2,model3,model4
0,cat,cat,dog,cat,dog
1,dog,dog,cat,cat,dog
2,dog,cat,cat,cat,dog
3,dog,dog,dog,cat,dog
4,cat,cat,cat,dog,dog
...,...,...,...,...,...
4995,dog,dog,dog,dog,dog
4996,dog,dog,cat,cat,dog
4997,dog,cat,cat,dog,dog
4998,cat,cat,cat,cat,dog


In [31]:
paws.actual.value_counts()

dog    3254
cat    1746
Name: actual, dtype: int64

In [32]:
paws['baseline'] = 'dog'

In [37]:
cols = paws.columns.drop('actual')

In [51]:
comparison_acc = []

for col in cols:
    comparison_acc.append(col + ' accuracy: ' + str((paws.actual == paws[col]).mean()))

In [52]:
comparison_acc

['model1 accuracy: 0.8074',
 'model2 accuracy: 0.6304',
 'model3 accuracy: 0.5096',
 'model4 accuracy: 0.7426',
 'baseline accuracy: 0.6508']

In [53]:
# model1 and model4 do beat the baseline in terms of accuracy

In [63]:
# precision
for col in cols:
    subset = paws[paws[col] == 'dog']
    print(f'{col} precision is: {(subset[col] == subset.actual).mean()}')

model1 precision is: 0.8900238338440586
model2 precision is: 0.8931767337807607
model3 precision is: 0.6598883572567783
model4 precision is: 0.7312485304490948
baseline precision is: 0.6508


In [64]:
# recall
subset = paws[paws.actual == 'dog']
for col in cols:
    print(f'{col} recall is: {(subset[col] == subset.actual).mean()}')

model1 recall is: 0.803318992009834
model2 recall is: 0.49078057775046097
model3 recall is: 0.5086047940995697
model4 recall is: 0.9557467732022127
baseline recall is: 1.0


In [68]:
# b. On dog pic team, what model to use for phaseI and phaseII?
# Would want to use recall for phaseI and precision for phaseII. So probably model4 and then model2. This would 
# have the machine 'catch' as many dogs as possible (without losing some to cat), and then the humans could ensure 
# that the actual photos uploaded were really dogs

# c. On cat pic team, what model to use for phaseI and phaseII?
# Would want to use precision for phaseI with model2 to ensure the lowest chance of assigning cats as dogs. For 
# phaseII would want to use recall to catch as many dogs as possible to ensure they don't get posted as cats, so
# model4.

In [73]:
for col in cols:
    print(f'{col} accuracy: {sklearn.metrics.accuracy_score(paws.actual, paws[col])}')

model1 accuracy: 0.8074
model2 accuracy: 0.6304
model3 accuracy: 0.5096
model4 accuracy: 0.7426
baseline accuracy: 0.6508


In [76]:
for col in cols:
    print(f'{col} precision: {sklearn.metrics.precision_score(paws.actual, paws[col], pos_label="dog")}')

model1 precision: 0.8900238338440586
model2 precision: 0.8931767337807607
model3 precision: 0.6598883572567783
model4 precision: 0.7312485304490948
baseline precision: 0.6508


In [77]:
for col in cols:
    print(f'{col} recall: {sklearn.metrics.recall_score(paws.actual, paws[col], pos_label="dog")}')

model1 recall: 0.803318992009834
model2 recall: 0.49078057775046097
model3 recall: 0.5086047940995697
model4 recall: 0.9557467732022127
baseline recall: 1.0


In [83]:
for col in cols:
    print(col)
    print(sklearn.metrics.classification_report(paws.actual, paws[col]))

model1
              precision    recall  f1-score   support

         cat       0.69      0.82      0.75      1746
         dog       0.89      0.80      0.84      3254

    accuracy                           0.81      5000
   macro avg       0.79      0.81      0.80      5000
weighted avg       0.82      0.81      0.81      5000

model2
              precision    recall  f1-score   support

         cat       0.48      0.89      0.63      1746
         dog       0.89      0.49      0.63      3254

    accuracy                           0.63      5000
   macro avg       0.69      0.69      0.63      5000
weighted avg       0.75      0.63      0.63      5000

model3
              precision    recall  f1-score   support

         cat       0.36      0.51      0.42      1746
         dog       0.66      0.51      0.57      3254

    accuracy                           0.51      5000
   macro avg       0.51      0.51      0.50      5000
weighted avg       0.55      0.51      0.52      5000